Ok, let's see what we have so far:

- `cropped-unique`: from 20200426 to 20200510, de-duplicated (roughly) using LSH.
- `labeled-first-batch`: from 20200426 to 20200502
- `labeled-first-batch`: from 20200503 to 20200510

For some reason, the model is showing generalization issues on new data.  I do not understand why this is happening. To debug, I'll create a new batch using randomly sampled images from 20200511 to 20200528. 20200529 is my test set (which is when I saw this issue first occuring when finally deployed to the Raspberry Pi).

In [44]:
from pathlib import Path

import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

In [6]:
df = pd.DataFrame({"batch": ["first", "second"], 
                   "BirdHome": [4222, 4829], 
                   "BirdRoaming": [1445, 1520]})

In [7]:
df.head()

,batch,BirdHome,BirdRoaming
0,first,4222,1445
1,second,4829,1520


In [8]:
df.sum(axis=0)

batch          firstsecond
BirdHome              9051
BirdRoaming           2965
dtype: object

In [13]:
df.sum(axis=0)[["BirdHome", "BirdRoaming"]].sum()

12016

In [30]:
RAW_PATH = Path("~/Birdcam/data/raw").expanduser()
K = 10000 # How many new ones to add.

DATE_START = 20200511
DATE_END = 20200528

In [34]:
jpeg_paths = []

for date in range(DATE_START, DATE_END + 1):
    path = RAW_PATH / str(date)
    if not path.exists():
        print(f"Error: {path} doesn't exist")
        break
    jpeg_paths.extend(list(path.glob("*.jpeg")))
    
jpeg_paths = sorted(jpeg_paths)
print(f"Found {len(jpeg_paths)} JPEGs")

Found 24563 JPEGs


In [40]:
rnd_jpeg_paths = np.random.choice(jpeg_paths, size=K, replace=False)

In [41]:
ROI_Y_MIN = 100
ROI_Y_MAX = 1080
ROI_X_MIN = 400
ROI_X_MAX = 1200

def apply_roi(img):
    return img[ROI_Y_MIN:ROI_Y_MAX, ROI_X_MIN:ROI_X_MAX]

In [45]:
DEST_PATH = RAW_PATH.parent / "to-be-labeled-third-batch"
DEST_PATH.mkdir()

In [46]:
for jpeg_path in tqdm(rnd_jpeg_paths):
    img = cv2.imread(str(jpeg_path))
    img_crop = apply_roi(img)
    cv2.imwrite(str(DEST_PATH / jpeg_path.name), img_crop, [cv2.IMWRITE_JPEG_QUALITY, 80])

100%|██████████| 10000/10000 [05:37<00:00, 29.59it/s]


N.b.: after labeling 5k, I assigned the remainder to batch 4.

Batch 4 is from `20200514T205401` to `20200523T065401`. Batch 3 is before and after those timestamps (I labeled some from the start and some from the end)